TensorFlowとは

TensorFlowはニューラルネットワークを構築するためのフレームワークの一種です。


以下のコードはバージョン1.13.0での動作を確認しています。


簡単な計算
簡単な計算をさせてみます。TensorFlowの基本的な流れです。

In [3]:
import tensorflow as tf
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)
sess = tf.Session()
output = sess.run(add)
print(output) # 12
sess.close()

12


In [4]:
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)
with tf.Session() as sess:
    output = sess.run(add)
    print(output) # 12

12


比較のために、NumPyでも同じ計算をさせてみます。

In [5]:
import numpy as np
a_n = np.array(5)
b_n = np.array(7)
output_n = np.add(a_n, b_n)
print(output_n) # 12

12


データフロープログラミング

TensorFlowの仕組みを見ていきます。TensorFlowではデータフローグラフを考えます。


簡単な計算の例では以下のようなデータフローグラフになります。

グラフはノード（丸い部分）とエッジ（線の部分）で表されます。ノードはオペレーション（操作）を表し、エッジはTensor（多次元配列）を表します。


今回の例では左側の灰色のノードでは定数を作るオペレーション（操作）、右の水色のノードでは足し算のオペレーション（操作）が行われています。エッジは整数値です。

TensorFlowの手順
TensorFlowの手順は以下のようになっています。


データフローグラフを構築する
データを入力して計算する

簡単な計算の例でそれぞれを解説します。


データフローグラフの構築
サンプルコードの以下の部分でデータフローグラフを構築しています。

In [6]:
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)

定数aと定数bを定義し、それらを足し算するデータフローグラフです。NumPyであれば3行で計算まで行いますが、TensorFlowでは実際の計算までは進みません。


それぞれをprintしてみても、エッジ（Tensor）の説明が返されるだけです。

In [7]:
print(a) # Tensor("Const:0", shape=(), dtype=int32)
print(add) # Tensor("Add:0", shape=(), dtype=int32)

Tensor("Const_4:0", shape=(), dtype=int32)
Tensor("Add_2:0", shape=(), dtype=int32)


データを入力して計算
データフローグラフの計算を行う際にはセッションという概念が登場します。


まずSessionオブジェクトを作成します。
そして、sess.run()の中にエッジ（Tensor）を入れます。そうすることで、そのエッジ（Tensor）がどのような値を持つかが返ってきます。

In [8]:
sess = tf.Session()
output = sess.run(add)
print(output) # 12

12


TensorFlowにおける値の扱い方
TensorFlowが値を扱う上で独自の概念として、placeholderとValiableがあります。


《placeholder》
placeholderはデータフローグラフの構築時には値が決まっていないものに使います。最初は配列の形だけ定義しておいて後から値を入れて使う空箱のような存在です。学習ごとに違う値が入る入力データや正解データなどに用いられます。

In [9]:
#サンプルコードをplaceholderで書き換え
c = tf.placeholder(tf.int32)
d = tf.placeholder(tf.int32)
add = tf.add(c, d)
sess = tf.Session()
output = sess.run(add, feed_dict={c:5, d:7})
print(output) # 12

12


セッションを実行する際に引数feed_dictを使い、placeholderに入れる値を辞書型で指定します。ここを書き換えることで異なる計算が可能になります。

In [10]:
output = sess.run(add, feed_dict={c:20, d:32})
print(output) # 52

52


今回の簡単な例では定数とplaceholderの違いは感じられませんが、ミニバッチ学習を行うような場合を想定すると、必要性がわかります。
？よくわかりません

TensorFlowにおける演算
サンプルコードではtf.add()を使用していますが、NumPyなどと同様に+を使用することも可能です。

In [11]:
add = a + b 
# tf.add(a, b)に等しい

セッションの終了
最後にセッションは終了させます。

In [12]:
sess.close()

セッションのインスタンス化から終了までに対して、with構文を使うことも可能です。

In [13]:
with tf.Session() as sess:
    sess.run() # ここに計算の実行コードを入れていく

TypeError: run() missing 1 required positional argument: 'fetches'

ロジスティック回帰の実装

TensorFlowを使いロジスティック回帰を実装していきます。入門1では単純な足し算でしたが、ここでは学習を伴う計算を行います。


論理回路
簡単な題材として、ロジスティック回帰による論理回路の再現を行います。論理回路は2つの値を入力し、1つの値を出力する関数のようなものです。入力も出力も0か1のみで表され、入力される組み合わせによって出力する値が変わります。


ANDゲートは入力された2つの値が両方とも1だった場合、出力が1となり、それ以外の組み合わせでは0を出力します。

データの作成
最初に学習用のトレーニングデータをNumPyにより作成しておきます。ANDゲートでは入力が2次元で出力が1次元となります。2つの入力が1のときだけ1を出力し、それ以外は0を出力するので以下のように定義できます。

In [14]:
import numpy as np
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])#[1,1]の時に[1]を出力する
y_train = np.array([[0],[0],[0],[1]])

データフローグラフの構築
まずはデータフローグラフを構築します。


学習データをTensorFlowのデータフローグラフに入力するための placeholder を用意しましょう。placeholderはデータフローグラフを作成する段階では値が決まっていない、空箱のような存在でした。
第一引数のtf.float32で行列要素の数値のデータ型を指定しています。第二引数の[None,2]で行列の形を指定しています。ここで定義されている2はデータの次元を表しています。Noneの部分はデータ数を表す部分です。今回のANDゲートの場合のデータ数は[0,0],[0,1],[1,0],[1,1]の4つしかないのでNoneの部分を[4,2]としても問題はありません。しかし、任意の数のデータを入れられるように、一般的にはNoneを使います。
？よくわかりません。

In [15]:
import tensorflow as tf
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

重みとバイアスの Valiable を用意します。Valiableとして用意するということは、これらが学習により更新を行う値であることを示します。
ここで、tf.Variable()の中でtf.zeros()という関数を呼び出していますが、初期値として0を入れているということです。

In [16]:
W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

次にモデルの出力y（＝仮定関数）と目的関数を定義します。
ロジスティック回帰の式は以下でした。ここでは正則化項は抜かしています。
$$
h_θ(x) = g(θ^T x).\\<br/>g(z) = \frac{1}{1+e^{−z}}.\\<br/>J(\theta)=  \frac{1}{m}  \sum_{i=1}^{m}[−y^{(i)} log(h_θ(x^{(i)})) − (1−y^{(i)}) log(1−h_θ(x^{(i)}))]
$$

これをTensorFlowで記述すると次のようになります。
tf.matmul()はNumPyにおけるnp.dot()に相当するベクトルの内積や、行列積を計算するためのメソッドです
なお、例えば回帰問題で二乗和誤差関数を使用するのであれば、tf.reduce_sum(tf.square(y - t))というように定義できます。

In [17]:
y = tf.sigmoid(tf.matmul(x, W) + b)
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

ここまでで、入力のための空箱であるplaceholderと学習可能なValiableをメソッドで結ぶことができました。
学習を行うために、勾配降下法を用いてパラメータを最適化するためのコードを加えます。目的関数をGradientDescentOptimizerに渡します。GradientDescentOptimizer()は引数で学習率を指定しています。


In [18]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

学習後の結果の正解が正しいかどうかの判定と正解率の計算もデータフローグラフとして定義できます。

In [19]:
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

1行目で結果が正解かどうか判定しています。一つ一つ見ていきましょう。まずtf.equal()は引数に指定された2つの値が等しいかどうかを判定してくれます。返り値はBool値です。tf.sign()は引数の値が正なら1、0なら0、負なら-1を返します。yが0.5以上かどうかで結果が決まるので、y-0.5とt-0.5の符号を比較しています。


2行目は正解率を計算するためのコードです。tf.reduce_mean()は多次元配列の各成分の平均を計算する関数です。tf.cast()でBool値を0,1に変換しています。つまりここでは正解で1、不正解で0と判定された配列の平均値をとっているので正解率を表していることになります。

データを入力して計算
セッションを準備してパラメータを最適化する計算を行います。

まずセッションのインスタンスを作成します。そして、tf.global_variables_initializer()によって上で定義したtf.Variable()の値(重み・バイアス)を初期化します。実行する際にはsess.run()を使います。

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [21]:
for epoch in range(1000):
    sess.run(train_step, feed_dict={x:x_train,t:y_train})
# 100回ごとに正解率を表示
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print ('epoch: %d, Accuracy: %f'
               %(epoch, acc_val))

epoch: 0, Accuracy: 0.750000
epoch: 100, Accuracy: 1.000000
epoch: 200, Accuracy: 1.000000
epoch: 300, Accuracy: 1.000000
epoch: 400, Accuracy: 1.000000
epoch: 500, Accuracy: 1.000000
epoch: 600, Accuracy: 1.000000
epoch: 700, Accuracy: 1.000000
epoch: 800, Accuracy: 1.000000
epoch: 900, Accuracy: 1.000000


2行目、sess.run()に上で定義したtrain_stepを入れることで、勾配降下法による学習を行っています。
8行目はsess.run()にaccuracyを入れることで、正解率を計算しています。計算結果がNumPy形式で返ってきているので、これをprintします。
形だけ定義していたplaceholderのxとtの中には値が何も入っていません。placeholderに値を設定するためにsess.run()のパラメータでfeed_dictを指定します。例えば、feed_dict={x:x_train,t:y_train})と書くことで空箱だったxにx_trainの値が入り、tにy_trainの値が入ります。
表示結果を見てみると、正解率が100%でうまく学習できているように見えます。

最後に各サンプルの計算結果を確認してロジスティック回帰の実装を終わります。先ほどのaccuracyと同様、実行することで計算結果が返ってくるためprintします。


In [22]:
#学習結果が正しいか確認
classified = sess.run(correct_prediction, feed_dict={
    x:x_train,
    t:y_train
})
#出力yの確認
prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
})
print(classified)
# [[ True]
# [ True]
# [ True]
# [ True]]
print(prob)
# [[  1.96514215e-04]
# [  4.90498319e-02]
# [  4.90498319e-02]
# [  9.31203783e-01]]

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9651403e-04]
 [4.9049813e-02]
 [4.9049813e-02]
 [9.3120378e-01]]


classifiedの結果は全てTrueで正しく学習されていることがわかります。probは上からほぼ[0,0,0,1]となっています。今回活性化関数に用いたのはシグモイド関数ですので出力yは確率として表示されています。上から3つは1になる確率がほぼ0％、一番下の1つは93％程度の確率で1になるということになります。


Wとbが学習後どのような値になっているかも見ておきましょう。Variableもsess.run()に入れることで値を確認できます。

In [23]:
print('W:', sess.run(W))
print('b:', sess.run(b))
# W: [[ 5.5699544]
# [ 5.5699544]]
# b: [-8.53457928]

W: [[5.569955]
 [5.569955]]
b: [-8.53458]


###  【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。


重みを初期化する必要があった
エポックのループが必要だった

損失関数を設定する必要があった  
誤差逆伝播法によって重みパラメータの勾配の計算  


それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。


データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。


Iris Species


目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。

【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

・重みの初期値はtf.Variable(tf.random_normal([ , ]))である。

In [1]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import tensorflow as tf

#iris_dataset= load_iris()
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50#重み1の列数、バイアス1の数、重み2の行数
n_hidden2 = 100#重み2の列数、バイアス2の数、重み３の行数
n_input = X_train.shape[1]#特徴量
n_samples = X_train.shape[0]#サンプル数
n_classes = 1#yは１列だから1、ラベルは分類の数だけある、予測値の列数(1)
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])#空箱、[データ数,次元数]
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#NNの適当な名前
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),#学習により更新を行う値
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    #tf.addは加算（matmalとb)
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    #tf.nn.reluで
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)#outputをlogitsで受け取る

# 目的関数、labels=ターゲット変数、logitsにoutputでlossを出す、tf.reduce_mean全ての要素を足し合わせて平均値を出力
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))#logitsはlayer_output、yハット
# 最適化手法、重みの更新の調整
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)#train_opはミニバッチのループ内で使う
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()
#↑データフローグラフの構築
# 計算グラフの実行(データを入力して計算)
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)#不明、訓練データのサンプル数をバッチサイズで割る
        total_loss = 0#なんのため？
        total_acc = 0#なんのため？
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

/Users/apple/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/apple/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/apple/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/apple/.py

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 0, loss : 1.8218, val_loss : 4.5825, acc : 0.750, val_acc : 0.688
Epoch 1, loss : 0.0036, val_loss : 2.2782, acc : 1.000, val_acc : 0.875
Epoch 2, loss : 0.0001, val_loss : 2.9416, acc : 1.000, val_acc : 0.750
Epoch 3, loss : 0.0110, val_loss : 3.2522, acc : 1.000, val_acc : 0.812
Epoch 4, loss : 0.0000, val_loss : 0.7246, acc : 1.000, val_acc : 0.812
Epoch 5, loss : 0.0005, val_loss : 2.8904, acc : 1.000, val_acc : 0.875
Epoch 6, loss : 0.0000, val_loss : 2.4210, acc : 1.000, val_acc : 0.812
Epoch 7, loss : 0.5958, val_loss : 5.2623, acc : 0.750, val_acc : 0.750
Epoch 8, loss : 0.0000, val_loss : 1.5183, acc : 1.000, val_acc : 0.812
Epoch 9, loss : 0.0687, val_loss : 5.0111, acc : 1.000, val_acc : 0.750
test_acc : 0.800


他のデータセットへの適用

これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。


Iris（3種類全ての目的変数を使用）
House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。

【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


Iris Species


2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。


《ヒント》


以下の2箇所は2クラス分類特有の処理です。
メソッドは以下のように公式ドキュメントを確認してください。

ロジスティック回帰は２クラス分類
Q.多クラス分類の処理を探す
tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow


tf.math.sign  |  TensorFlow


＊tf.signとtf.math.signは同じ働きをします。要素ごとに正なら1、0なら0、負なら-1となる変換をかける。
何のため？

loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

シグモイド二値分類
ソフトマックスに変えるのはなぜ

 tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
 クロスエントロピーは多クラス分類でよく使われる誤差関数の一つで、dd番目のデータに対して、クラスiiである推定確率xd,ixd,iと真の確率yd,iyd,iの誤差を計算します。
 クロスエントロピーについてはTensorFlowでは単体の関数は用意されていないようなので、ソフトマックス関数とクロスエントロピーをセットにした、tf.nn.softmax_cross_entropy_with_logits_v2やtf.nn.sparse_softmax_cross_entropy_with_logitsを使います。"with_logits"というのは、この関数は内部でソフトマックスも計算するから、ニューラルネットワークの出力をそのまま入力してね、という意味です。

correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))



In [25]:
from sklearn.preprocessing import OneHotEncoder

In [57]:
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出

#df = df[df]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis]

#yのOnehot化
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)#インスタンス化
y = enc.fit_transform(y)#fit_transform、データを関数に適合させるだけではなく、形も変形する
#fitはデータに関数を適合させる、真の関数に近似する関数を作る

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [60]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):#目的？
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.1
batch_size = 10
num_epochs = 100
n_hidden1 = 50#重み1の列数、バイアス1の数、重み2の行数
n_hidden2 = 100#重み2の列数、バイアス2の数、重み３の行数
n_input = X_train.shape[1]#特徴量
n_samples = X_train.shape[0]#サンプル数
n_classes = 3#yは１列だから1、ラベルは分類の数だけある、予測値の列数(1)
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])#空箱、[データ数,次元数]
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#NNの適当な名前
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),#学習により更新を行う値
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    #tf.addは加算（matmalとb)
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    #tf.nn.reluで
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)#outputをlogitsで受け取る

# 目的関数、labels=ターゲット変数、logitsにoutputでlossを出す、tf.reduce_mean全ての要素を足し合わせて平均値を出力
#loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))#logitsはlayer_output、yハット
loss_op = tf.reduce_mean( tf.losses.softmax_cross_entropy(onehot_labels=Y, logits=logits)) 
#with_logits"というのは、この関数は内部でソフトマックスも計算するから、ニューラルネットワークの出力をそのまま入力
# 最適化手法、重みの更新の調整
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)#train_opはミニバッチのループ内で使う
# 推定結果
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))#(tf.sigmoid(logits) - 0.5))
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1)) #遠藤さん、何をしている？何番目のアイリスかを比較している
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()
#↑データフローグラフの構築
# 計算グラフの実行(データを入力して計算)
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)#不明、訓練データのサンプル数をバッチサイズで割る
        total_loss = 0#なんのため？
        total_acc = 0#なんのため？
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):#ミニバッチ
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 102.8792, val_loss : 31.0601, acc : 0.500, val_acc : 0.458
Epoch 1, loss : 8.7579, val_loss : 8.4823, acc : 0.800, val_acc : 0.833
Epoch 2, loss : 1.9219, val_loss : 2.7885, acc : 0.890, val_acc : 0.875
Epoch 3, loss : 1.7209, val_loss : 1.5249, acc : 0.910, val_acc : 0.917
Epoch 4, loss : 0.8113, val_loss : 3.4528, acc : 0.960, val_acc : 0.833
Epoch 5, loss : 1.1159, val_loss : 0.8146, acc : 0.940, val_acc : 0.958
Epoch 6, loss : 0.6522, val_loss : 0.0861, acc : 0.960, val_acc : 0.958
Epoch 7, loss : 0.5404, val_loss : 1.8063, acc : 0.970, val_acc : 0.875
Epoch 8, loss : 1.0847, val_loss : 0.5066, acc : 0.920, val_acc : 0.958
Epoch 9, loss : 0.7439, val_loss : 0.9573, acc : 0.960, val_acc : 0.917
Epoch 10, loss : 0.2575, val_loss : 0.4853, acc : 0.960, val_acc : 0.958
Epoch 11, loss : 0.1783, val_loss : 0.2291, acc : 0.970, val_acc : 0.958
Epoch 12, loss : 0.1876, val_loss : 0.3406, acc : 0.970, val_acc : 0.958
Epoch 13, loss : 0.1811, val_loss : 0.3326, acc : 0.970, v

In [64]:
#遠藤さん
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.1
batch_size = 10
num_epochs = 5
n_hidden1 = 50#重み1の列数、バイアス1の数、重み2の行数
n_hidden2 = 100#重み2の列数、バイアス2の数、重み３の行数
n_input = X_train.shape[1]#特徴量
n_samples = X_train.shape[0]#サンプル数
n_classes = 3#yは１列だから1、ラベルは分類の数だけある、予測値の列数(1)
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])#空箱、[データ数,次元数]
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#NNの適当な名前
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),#学習により更新を行う値
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    #tf.addは加算（matmalとb)
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    #tf.nn.reluで
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数(3値分類)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
#correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1)) #遠藤さん、tf.argmax()： 第2パラメーターに1をセット、行ごとに最大となる列を返す
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5)) #元々
#correct_pred = tf.equal(tf.math.argmax(Y), tf.math.argmax(logits))
# correct_pred = tf.argmax(tf.nn.softmax(logits), axis=1)
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))#同じ

# variableの初期化
init = tf.global_variables_initializer()
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)#トータルバッチ
        loss_list = []
        acc_list = []
        val_loss_list = []
        acc_val_list = []
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})#不明
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            val_loss, val_acc, pred = sess.run([loss_op, accuracy,correct_pred], feed_dict={X: X_val, Y: y_val})
            loss_list.append(loss)
            acc_list.append(acc)
            val_loss_list.append(val_loss)
            acc_val_list.append(val_acc)
            print('あ',pred)
        print("Epoch {}, total_loss : {:.4f}, val_loss : {:.4f}, total_acc : {:.3f}, val_acc : {:.3f}".format(
            epoch, np.mean(loss_list), np.mean(val_loss_list), np.mean(acc_list), np.mean(acc_val_list)))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

あ [[ True  True  True]
 [False False  True]
 [False False  True]
 [ True  True  True]
 [False False  True]
 [ True False False]
 [ True False False]
 [ True False False]
 [ True False False]
 [False False  True]
 [False False  True]
 [ True  True  True]
 [ True  True  True]
 [False False  True]
 [ True False False]
 [ True False False]
 [ True  True  True]
 [False False  True]
 [ True False False]
 [ True False False]
 [ True  True  True]
 [False False  True]
 [False False  True]
 [ True False False]]
あ [[False  True  True]
 [ True False  True]
 [ True False  True]
 [False  True  True]
 [ True False  True]
 [False False False]
 [False False False]
 [False False False]
 [False False False]
 [ True False  True]
 [ True False  True]
 [False  True  True]
 [False  True  True]
 [ True False  True]
 [False False False]
 [False False False]
 [False  True  True]
 [ True False  True]
 [False False False]
 [False False False]
 [False  True  True]
 [ True False  True]
 [ True False  True]
 [False 

test_acc : 0.678


【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。



In [55]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# データセットの読み込み
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["SalePrice"]
X = df.loc[:  , ["GrLivArea" , "YearBuilt"]]
y = np.array(y)
X = np.array(X)
# 正則化
# ss = StandardScaler()
# ss.fit(X)
# X = ss.transform(X)
#対数変換
y = np.log(y)
X = np.log(X)
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 1e-3
batch_size = 10
num_epochs = 10

n_hidden1 = 10
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# 損失関数を指定(
# L2損失関数を指定(RMSE)
#loss = tf.sqrt(tf.reduce_mean(tf.square(y_target - model_output)))
loss_op = tf.sqrt(tf.reduce_mean(tf.square(Y - logits)))
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# correct_pred = tf.equal(tf.argmax(Y ,1), tf.argmax(logits , 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)#不明、訓練データのサンプル数をバッチサイズで割る
        total_loss = 0#なんのため？
        total_acc = 0#なんのため？
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
# with tf.Session() as sess:
#     sess.run(init)
#     for epoch in range(num_epochs):
#         # エポックごとにループ
#         total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
#         total_loss = 0
#         total_acc = 0
#         for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
#             # ミニバッチごとにループ
#             sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
#             loss = sess.run(loss_op , feed_dict={X: mini_batch_x, Y: mini_batch_y})
#             total_loss += loss
#             total_acc += acc
#         total_loss /= n_samples
#         total_acc /= n_samples
#         val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
#         print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, loss, val_loss))
#         test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
#         print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 347.0931, val_loss : 139.8113, acc : 1.000, val_acc : 1.000
Epoch 1, loss : 25.7954, val_loss : 0.8124, acc : 0.973, val_acc : 1.000
Epoch 2, loss : 0.7703, val_loss : 0.5565, acc : 1.000, val_acc : 1.000
Epoch 3, loss : 0.5875, val_loss : 0.7181, acc : 1.000, val_acc : 1.000
Epoch 4, loss : 0.4800, val_loss : 0.6520, acc : 1.000, val_acc : 1.000
Epoch 5, loss : 0.4858, val_loss : 0.4358, acc : 1.000, val_acc : 1.000
Epoch 6, loss : 0.4623, val_loss : 0.4130, acc : 1.000, val_acc : 1.000
Epoch 7, loss : 0.4974, val_loss : 0.3993, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.4567, val_loss : 0.4197, acc : 1.000, val_acc : 1.000
Epoch 9, loss : 0.4779, val_loss : 0.4166, acc : 1.000, val_acc : 1.000
test_acc : 1.000


【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [43]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1 , 784)
X_test = X_test.reshape(-1 , 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
# y_train = tf.one_hot(y_train , depth=3 ,dtype=None)
# y_test = tf.one_hot(y_test , depth=3 , dtype=None)
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.transform(y_test[:, np.newaxis])


# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])

Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y ,1), tf.argmax(logits , 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.6114, val_loss : 1.1027, acc : 0.800, val_acc : 0.717
Epoch 1, loss : 0.4930, val_loss : 0.7755, acc : 0.800, val_acc : 0.824
Epoch 2, loss : 0.2462, val_loss : 0.4959, acc : 0.900, val_acc : 0.894
Epoch 3, loss : 0.1886, val_loss : 0.3368, acc : 0.900, val_acc : 0.916
Epoch 4, loss : 0.1018, val_loss : 0.3349, acc : 0.900, val_acc : 0.925
Epoch 5, loss : 0.0140, val_loss : 0.3058, acc : 1.000, val_acc : 0.930
Epoch 6, loss : 0.0331, val_loss : 0.3364, acc : 1.000, val_acc : 0.932
Epoch 7, loss : 0.0023, val_loss : 0.3067, acc : 1.000, val_acc : 0.931
Epoch 8, loss : 0.0110, val_loss : 0.2885, acc : 1.000, val_acc : 0.931
Epoch 9, loss : 0.0500, val_loss : 0.3229, acc : 1.000, val_acc : 0.931
test_acc : 0.926
